# Lab 1: Airflow Basics - Giới thiệu Apache Airflow

## 🎯 Objectives
- Hiểu kiến trúc và components của Apache Airflow
- Học cách sử dụng Airflow Web UI
- Làm quen với Airflow CLI commands
- Tạo và chạy DAG đầu tiên

## 📋 Prerequisites
- Airflow cluster đang chạy (`docker compose up -d`)
- Truy cập được Airflow UI tại http://localhost:8080
- Basic understanding của Python và workflows

## 🏗️ Airflow Architecture Overview
```
┌─────────────┐
│  Web UI     │  Port 8080 - Quản lý và giám sát DAGs
└──────┬──────┘
      │
┌─────▼──────┐
│ Scheduler  │  Lập lịch và trigger tasks
└─────┬──────┘
      │
┌─────▼──────┐
│ Executor   │  Thực thi tasks (LocalExecutor)
└─────┬──────┘
      │
┌─────▼──────┐
│ Metadata   │  PostgreSQL - Lưu trữ metadata
└────────────┘
```

## 📚 Key Concepts
- **DAG (Directed Acyclic Graph)**: Workflow được định nghĩa bằng Python
- **Task**: Đơn vị công việc nhỏ nhất trong DAG
- **Operator**: Template cho một task (BashOperator, PythonOperator, etc.)
- **Scheduler**: Component lập lịch và trigger DAG runs
- **Executor**: Component thực thi tasks


## 1. Kiểm tra Airflow Installation


In [ ]:
# Kiểm tra Airflow version và connection
import subprocess
import requests
import json

# Kiểm tra Airflow CLI
try:
    result = subprocess.run(
        ["docker", "compose", "exec", "-T", "airflow-webserver", "airflow", "version"],
        capture_output=True,
        text=True,
        timeout=10
    )
    print("✅ Airflow CLI accessible")
    print(result.stdout)
except Exception as e:
    print(f"⚠️  Airflow CLI check failed: {e}")
    print("💡 Make sure Airflow is running: docker compose up -d")

# Kiểm tra Airflow UI
try:
    response = requests.get("http://localhost:8080/health", timeout=5)
    if response.status_code == 200:
        print("\n✅ Airflow Web UI is accessible at http://localhost:8080")
        print(f"   Health check: {response.json()}")
    else:
        print(f"\n⚠️  Airflow UI returned status code: {response.status_code}")
except Exception as e:
    print(f"\n⚠️  Cannot connect to Airflow UI: {e}")
    print("💡 Make sure Airflow webserver is running: docker compose up -d")


## 2. Liệt kê DAGs


In [ ]:
# Liệt kê tất cả DAGs
result = subprocess.run(
    ["docker", "compose", "exec", "-T", "airflow-webserver", "airflow", "dags", "list"],
    capture_output=True,
    text=True,
    timeout=10
)

print("📋 Available DAGs:")
print("=" * 60)
print(result.stdout)
print("=" * 60)

# Parse và hiển thị DAGs
lines = result.stdout.strip().split('\n')
dag_list = []
for line in lines[2:]:  # Skip header lines
    if line.strip():
        parts = line.split()
        if len(parts) >= 2:
            dag_list.append({
                'dag_id': parts[0],
                'owner': parts[1] if len(parts) > 1 else 'N/A',
                'status': parts[-1] if len(parts) > 2 else 'N/A'
            })

if dag_list:
    print(f"\n✅ Found {len(dag_list)} DAG(s):")
    for dag in dag_list:
        print(f"   - {dag['dag_id']} (Owner: {dag['owner']})")
else:
    print("\n⚠️  No DAGs found. Make sure DAGs are in the dags/ directory.")


## 3. Xem chi tiết một DAG


In [ ]:
# Xem chi tiết DAG hello_world
dag_id = "hello_world"

print(f"📊 DAG Details: {dag_id}")
print("=" * 60)

# Show DAG structure
result = subprocess.run(
    ["docker", "compose", "exec", "-T", "airflow-webserver", "airflow", "dags", "show", dag_id],
    capture_output=True,
    text=True,
    timeout=10
)

if result.returncode == 0:
    print(result.stdout)
else:
    print(f"⚠️  DAG '{dag_id}' not found or error occurred")
    print(result.stderr)
    print("\n💡 Available DAGs:")
    subprocess.run(
        ["docker", "compose", "exec", "-T", "airflow-webserver", "airflow", "dags", "list"],
        timeout=10
    )


## 4. Sử dụng Airflow REST API

Airflow cung cấp REST API để tương tác programmatically. Chúng ta sẽ sử dụng API để:
- Lấy thông tin DAGs
- Trigger DAG runs
- Xem task status


In [ ]:
# Cấu hình Airflow API
AIRFLOW_URL = "http://localhost:8080"
AIRFLOW_USERNAME = "airflow"
AIRFLOW_PASSWORD = "airflow"

# Helper function để gọi API
def airflow_api_call(endpoint, method="GET", data=None):
    """Gọi Airflow REST API"""
    url = f"{AIRFLOW_URL}/api/v1/{endpoint}"
    auth = (AIRFLOW_USERNAME, AIRFLOW_PASSWORD)
    
    try:
        if method == "GET":
            response = requests.get(url, auth=auth, timeout=10)
        elif method == "POST":
            response = requests.post(url, auth=auth, json=data, timeout=10)
        else:
            raise ValueError(f"Unsupported method: {method}")
        
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ API call failed: {e}")
        return None

# Lấy danh sách DAGs từ API
print("📋 Getting DAGs from REST API...")
dag_list_response = airflow_api_call("dags")

if dag_list_response:
    dags = dag_list_response.get("dags", [])
    print(f"\n✅ Found {len(dags)} DAG(s) via API:")
    for dag in dags[:10]:  # Show first 10
        print(f"   - {dag['dag_id']} (Is Paused: {dag.get('is_paused', False)})")
    
    if len(dags) > 10:
        print(f"   ... and {len(dags) - 10} more")


## 5. Trigger DAG Run

Chúng ta có thể trigger một DAG run thông qua REST API hoặc CLI.


In [ ]:
# Trigger DAG run qua REST API
dag_id = "hello_world"

print(f"🚀 Triggering DAG: {dag_id}")

# Trigger DAG
trigger_response = airflow_api_call(
    f"dags/{dag_id}/dagRuns",
    method="POST",
    data={
        "dag_run_id": f"manual_run_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}",
        "conf": {}
    }
)

if trigger_response:
    print(f"✅ DAG run triggered successfully!")
    print(f"   DAG Run ID: {trigger_response.get('dag_run_id')}")
    print(f"   State: {trigger_response.get('state')}")
    print(f"\n💡 Check the Airflow UI to see the DAG run: http://localhost:8080")
else:
    print("⚠️  Failed to trigger DAG run")


## 6. Xem DAG Runs và Task Status


In [ ]:
import pandas as pd
import time

dag_id = "hello_world"

# Đợi một chút để DAG run được tạo
print("⏳ Waiting for DAG run to be created...")
time.sleep(2)

# Lấy danh sách DAG runs
print(f"\n📊 Getting DAG runs for: {dag_id}")
dag_runs_response = airflow_api_call(f"dags/{dag_id}/dagRuns?limit=5")

if dag_runs_response:
    dag_runs = dag_runs_response.get("dag_runs", [])
    
    if dag_runs:
        print(f"\n✅ Found {len(dag_runs)} DAG run(s):")
        print("=" * 80)
        
        for run in dag_runs:
            print(f"\nDAG Run ID: {run.get('dag_run_id')}")
            print(f"  State: {run.get('state')}")
            print(f"  Start Date: {run.get('start_date')}")
            print(f"  End Date: {run.get('end_date', 'N/A')}")
            print(f"  Duration: {run.get('duration', 'N/A')} seconds")
        
        # Lấy task instances cho DAG run mới nhất
        latest_run_id = dag_runs[0]['dag_run_id']
        print(f"\n📋 Task instances for DAG run: {latest_run_id}")
        
        tasks_response = airflow_api_call(
            f"dags/{dag_id}/dagRuns/{latest_run_id}/taskInstances"
        )
        
        if tasks_response:
            tasks = tasks_response.get("task_instances", [])
            print(f"\n✅ Found {len(tasks)} task instance(s):")
            for task in tasks:
                print(f"   - {task.get('task_id')}: {task.get('state')}")
    else:
        print("⚠️  No DAG runs found")
        print("💡 Trigger a DAG run first or wait for scheduled runs")


## 7. Tóm tắt và Next Steps

### ✅ Những gì đã học:
1. Kiến trúc cơ bản của Airflow
2. Cách sử dụng Airflow CLI
3. Cách sử dụng Airflow REST API
4. Cách trigger và monitor DAG runs

### 📚 Next Lab:
- **Lab 2**: DAGs và Tasks - Học cách tạo DAGs với Task SDK
- Tạo DAGs với @dag và @task decorators
- Định nghĩa task dependencies
- Xử lý errors và retries

### 🔗 Useful Links:
- [Airflow Documentation](https://airflow.apache.org/docs/apache-airflow/3.1.1/)
- [Airflow Task SDK](https://airflow.apache.org/docs/apache-airflow/3.1.1/task-sdk/index.html)
- [Airflow REST API](https://airflow.apache.org/docs/apache-airflow/3.1.1/stable-rest-api-ref.html)
